<a href="https://colab.research.google.com/github/LKNera/Jlptn5/blob/main/AvatarNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

neranjanawijethunga_avatar_path = kagglehub.dataset_download('neranjanawijethunga/avatar')

print('Data source import complete.')


In [1]:
# Improved Digital Human Generation Workflow
# Runs on Google Colab or Kaggle with GPU enabled.
# Changes:
# - Use Avatar.mp4 as source video for MuseTalk instead of generated image.
# - Conversation is continuous with memory (chat_history).
# - Loads all necessary items first, then starts Gradio interface for ongoing conversation until user stops.
# - For real-time approximation, use dynamic batch and efficient processing; generation per turn but fast.
# - Automate downloads, progress bars.
from tqdm import tqdm
import os
import subprocess
import sys

# Install necessary libraries, fixing incompatible versions
# Pinning versions of conflicting libraries based on the error messages
!{sys.executable} -m pip install diffusers transformers==4.39.2 accelerate torch torchaudio soundfile parler-tts gradio moviepy tqdm huggingface_hub aiortc websockets --quiet
!{sys.executable} -m pip install --no-cache-dir -U openmim --quiet
!{sys.executable} -m mim install mmengine --quiet
!{sys.executable} -m mim install mmcv==2.0.1 --quiet
!{sys.executable} -m mim install mmdet==3.1.0 --quiet
!{sys.executable} -m mim install mmpose==1.1.0 --quiet
!apt update -qq && apt install ffmpeg -y -qq
!{sys.executable} -m pip install git+https://github.com/huggingface/parler-tts.git --quiet
!{sys.executable} -m pip install --upgrade transformers==4.39.2 --quiet # Ensure transformer version consistency


import torch
import numpy as np
import asyncio
import tempfile
import warnings
from typing import List
import gradio as gr
import soundfile as sf
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoTokenizer as HFTokenizer
from huggingface_hub import hf_hub_download

import time
import moviepy.editor as mpy
warnings.filterwarnings("ignore")

os.environ["FFMPEG_PATH"] = "/usr/bin/ffmpeg"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.7/100.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
import shutil
shutil.copy("/kaggle/input/avatar/Avatar IV Video.mp4","Avatar.mp4")

'Avatar.mp4'

In [ ]:
# Step 1: Clone and setup MuseTalk repo automatically
print("Step 1: Setting up MuseTalk...")
if not os.path.exists("MuseTalk"):
    subprocess.run(["git", "clone", "https://github.com/TMElyralab/MuseTalk.git"], check=True, stdout=subprocess.DEVNULL)
os.chdir("MuseTalk")
subprocess.run(["pip", "install", "-r", "requirements.txt", "--quiet"], check=True, stdout=subprocess.DEVNULL)
# Download models using the provided script
subprocess.run(["sh", "download_weights.sh"], check=True)
os.chdir("..")
print("MuseTalk setup complete.")

Step 1: Setting up MuseTalk...


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
parler-tts 0.2.2 requires transformers<=4.46.1,>=4.46.1, but you have transformers 4.39.2 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
datasets 3.6.0 requires requests>=2.32.2, but you have requests 2.28.2 which is incompatible.
datasets 3.6.0 requires tqdm>=4.66.3, but you have tqdm 4.65.2 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.2 which is incompatible.


  Using cached huggingface_hub-0.35.1-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.35.1-py3-none-any.whl (563 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.30.2
    Uninstalling huggingface-hub-0.30.2:
      Successfully uninstalled huggingface-hub-0.30.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
parler-tts 0.2.2 requires transformers<=4.46.1,>=4.46.1, but you have transformers 4.39.2 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
datasets 3.6.0 requires requests>=2.32.2, but you have requests 2.28.2 which is incompatible.
datasets 3.6.0 requires tqdm>=4.66.3, but you have tqdm 4.65.2 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.2 which is incompatible.


⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.


...

Fetching 1 files:   0%|          | 0/1 [00:02<?, ?it/s]
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 61, in main
    service.run()
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py", line 157, in run
    print(self._download())  # Print path to downloaded files
          ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py", line 191, in _download
    return snapshot_download(
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/_snapshot_download.py", line 332, in snapshot_download
    thread_map(
  File "

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.


Fetching 1 files:   0%|          | 0/1 [00:02<?, ?it/s]
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 61, in main
    service.run()
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py", line 157, in run
    print(self._download())  # Print path to downloaded files
          ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py", line 191, in _download
    return snapshot_download(
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/_snapshot_download.py", line 332, in snapshot_download
    thread_map(
  File "

✅ All weights have been downloaded successfully!
MuseTalk setup complete.


100 44.6M  100 44.6M    0     0   155M      0 --:--:-- --:--:-- --:--:--  155M


In [ ]:
# Assume Avatar.mp4 is in working directory
if not os.path.exists("Avatar.mp4"):
    raise FileNotFoundError("Avatar.mp4 not found in working directory.")

In [ ]:
# Upgrade transformers to latest from git to fix module imports
subprocess.run(["pip", "install", "git+https://github.com/huggingface/transformers", "--quiet"], check=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
parler-tts 0.2.2 requires transformers<=4.46.1,>=4.46.1, but you have transformers 4.57.0.dev0 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
datasets 3.6.0 requires requests>=2.32.2, but you have requests 2.28.2 which is incompatible.
datasets 3.6.0 requires tqdm>=4.66.3, but you have tqdm 4.65.2 which is incompatible.


CompletedProcess(args=['pip', 'install', 'git+https://github.com/huggingface/transformers', '--quiet'], returncode=0)

In [ ]:
!pip install transformers

In [2]:
# Removed as it is not needed for this workflow and causes import errors.

In [ ]:
# Step 2: Setup local TTS (Parler-TTS)
print("Step 2: Setting up Parler-TTS...")
device = "cuda" if torch.cuda.is_available() else "cpu"
parler_model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler_tts_mini_v0.1").to(device)
parler_tokenizer = HFTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")
print("Parler-TTS ready.")

Step 2: Setting up Parler-TTS...


RuntimeError: Failed to import transformers.models.mllama.configuration_mllama because of the following error (look up to see its traceback):
No module named 'transformers.models.mllama.configuration_mllama'

In [ ]:
# Step 3: Setup local LLM (DialoGPT-medium)
print("Step 3: Setting up LLM...")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.pad_token = tokenizer.eos_token
llm_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)
chat_history_ids = None
print("LLM ready.")

# Step 4: Dynamic batch size for MuseTalk
def get_dynamic_batch_size():
    if not torch.cuda.is_available():
        return 1
    total_mem = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # GB
    used_mem = torch.cuda.memory_allocated() / (1024 ** 3)
    free_mem = total_mem - used_mem
    if free_mem > 16:
        return 16
    elif free_mem > 8:
        return 8
    elif free_mem > 4:
        return 4
    else:
        return 2

Step 3: Setting up LLM...


RuntimeError: Failed to import transformers.models.dbrx.configuration_dbrx because of the following error (look up to see its traceback):
No module named 'transformers.models.dbrx.configuration_dbrx'

In [ ]:
# Moved function definitions to cell yV2GDPTrtZbQ and removed pyttsx3

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'requires' from 'transformers.utils.import_utils' (/usr/local/lib/python3.11/dist-packages/transformers/utils/import_utils.py)

In [ ]:
# Step 5: Animate with MuseTalk (async, in-memory mux, use Avatar.mp4)
async def animate_with_musetalk(video_path: str, audio_path: str, audio_arr: np.ndarray):
    batch_size = get_dynamic_batch_size()
    os.chdir("MuseTalk")

    # Temp config
    config_content = f"""
mode: normal
batch_size: {batch_size}
skip_save_images: true
result_dir: /tmp/musetalk_results
video_path: {os.path.abspath(video_path)}  # Use Avatar.mp4
audio_path: {os.path.abspath(audio_path)}
unet_model_path: models/musetalk/pytorch_model.bin
unet_config: models/musetalk/musetalk.json
vae_model_path: models/sd-vae-ft-mse
face_detection_path: models/face_detection
dwpose_model_path: models/dwpose/dw-ll_ucoco_384.pth
version: v1
ffmpeg_path: {os.environ['FFMPEG_PATH']}
    """
    config_path = "/tmp/musetalk_config.yaml"
    with open(config_path, "w") as f:
        f.write(config_content)

    # Create result dir
    os.makedirs("/tmp/musetalk_results", exist_ok=True)

    # Run inference with progress
    cmd = ["python", "-m", "scripts.inference", "--inference_config", config_path]
    with tqdm(total=1, desc="Running MuseTalk inference") as pbar:
        proc = await asyncio.create_subprocess_exec(*cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
        stdout, stderr = await proc.communicate()
        pbar.update(1)
    if proc.returncode != 0:
        raise RuntimeError(f"MuseTalk failed: {stderr.decode()}")

    # Output video (no audio)
    generated_video = "/tmp/musetalk_results/generated_video.mp4"
    if not os.path.exists(generated_video):
        raise FileNotFoundError("MuseTalk output not found")

    # In-memory mux with moviepy
    video_clip = mpy.VideoFileClip(generated_video)
    # Convert audio_arr to in-memory audio clip
    with tempfile.NamedTemporaryFile(suffix=".wav") as tmp_audio:
        sf.write(tmp_audio.name, audio_arr, 16000)
        audio_clip = mpy.AudioFileClip(tmp_audio.name)

    final_clip = video_clip.set_audio(audio_clip)

    # Write to temp file
    with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as tmp:
        output_path = tmp.name
    final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac', logger=None)

    # Cleanup
    video_clip.close()
    audio_clip.close()
    final_clip.close()
    try:
        os.remove(generated_video)
        os.remove(audio_path)
    except:
        pass

    os.chdir("..")
    return output_path

In [ ]:
# Step 6: Gradio Interface (continuous conversation with memory)
print("Step 6: Launching interface...")
chat_history = []

# Moved generate_tts and generate_response definitions here
def generate_tts(text, output_path="temp_audio.wav"):
    description = "A female speaker with a clear and natural voice."
    input_ids = parler_tokenizer(description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = parler_tokenizer(text, return_tensors="pt").input_ids.to(device)

    with torch.no_grad(), tqdm(total=1, desc="Generating TTS") as pbar:
        generation = parler_model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
        pbar.update(1)
    audio_arr = generation.cpu().numpy().squeeze()
    samplerate = parler_model.config.sampling_rate
    sf.write(output_path, audio_arr, samplerate)
    return output_path, audio_arr, samplerate

def generate_response(message):
    global chat_history_ids
    new_user_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt').to(device)

    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
    else:
        bot_input_ids = new_user_input_ids

    with tqdm(total=1, desc="Generating response") as pbar:
        chat_history_ids = llm_model.generate(
            bot_input_ids,
            max_length=1000,
            num_return_sequences=1,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
        pbar.update(1)

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response.strip()


async def chat_interface(message, history):
    if not message:
        return history, "Avatar.mp4"  # Fallback to source video

    try:
        # Generate response (with memory)
        response = generate_response(message)

        # Generate TTS
        audio_path, audio_arr, samplerate = generate_tts(response) # Added samplerate

        # Animate
        video_path = await animate_with_musetalk("Avatar.mp4", audio_path, audio_arr)

        # Update history
        history.append([message, response])

        return history, video_path
    except Exception as e:
        print(f"Error: {e}")
        return history, "Avatar.mp4"  # Fallback

with gr.Blocks(title="Local Digital Human Chat") as demo:
    gr.Markdown("# Local Digital Human Workflow using Avatar.mp4")
    gr.Markdown("Chat with animated character using local models. Conversation is continuous.")

    chatbot = gr.Chatbot(height=400)
    msg_input = gr.Textbox(placeholder="Type message...", label="Input")
    video_output = gr.Video(label="Animation", autoplay=True)
    clear_btn = gr.Button("Clear")

    msg_input.submit(
        lambda msg, hist: asyncio.run(chat_interface(msg, hist)),
        inputs=[msg_input, chatbot],
        outputs=[chatbot, video_output]
    ).then(
        lambda: gr.update(value=""),
        outputs=msg_input
    )

    clear_btn.click(lambda: ([], "Avatar.mp4"), outputs=[chatbot, video_output])

    video_output.value = "Avatar.mp4"  # Initial source video

demo.launch(share=True, debug=True)

Step 6: Launching interface...
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c3bc9124e7d1402963.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Error: name 'generate_response' is not defined
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c3bc9124e7d1402963.gradio.live
